# Dhruv Rana #

In [36]:
import torch
import numpy as np
import tensorflow as tf
from keras.applications.imagenet_utils import preprocess_input,decode_predictions
from keras.applications.resnet import ResNet50

In [37]:
resenet50 = tf.keras.applications.resnet.ResNet50()

In [38]:
img = tf.keras.preprocessing.image.load_img('golfcart1.jpg',target_size=(244,244))
img_arr = tf.keras.preprocessing.image.img_to_array(img)
img_batch = np.expand_dims(img_arr,axis=0)


In [39]:
img_processed = tf.keras.applications.resnet.preprocess_input(img_batch)

predict = resenet50.predict(img_processed)

predict_class = decode_predictions(predict)

ValueError: in user code:

    File "C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\e

In [46]:
from keras.utils import load_img, img_to_array
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = 'golfcart1.jpg'
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 1s 582ms/step
Predicted: [('n03445924', 'golfcart', 0.9907712), ('n03384352', 'forklift', 0.009174021), ('n03649909', 'lawn_mower', 1.6243515e-05)]
